5 Underrated Pandas Techniques to Make EDA Easier
===

This notebook demonstrates the code examples from this [article](https://tobeadatascientist.substack.com/p/5-underrated-pandas-techniques-to-make-eda-easier), showcasing the before and after of each technique.

For more resources like this, visit [tobeadatascientist.com](https://tobeadatascientist.com)

# Import data

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/andresvourakis/tbds-demos/refs/heads/main/5-underrated-pandas-techniques/data.csv')

In [3]:
df.head()

,order_id,customer_id,order_date,region,category,quantity,price_per_unit,tags,gender,total_sales
0,1,100,2025-01-10,North,Clothing,2,67,"['premium', 'gift']",Non-binary,10000
1,2,104,2025-01-12,West,Clothing,2,139,"['premium', 'gift']",Female,5
2,3,109,2025-01-14,North,Electronics,5,170,"['premium', 'gift']",Non-binary,15000
3,4,101,2025-01-16,East,Home Decor,5,158,['discounted'],Male,790
4,5,102,2025-01-18,West,Home Decor,2,112,"['premium', 'gift']",Female,224


**This is an e-commerce sales dataset**

Here is what's included:
- order_id: Unique identifier for each order.
- customer_id: Unique identifier for customers.
- order_date: Date of the order.
- region: Region where the order was placed (e.g., North, South, East, West).
- category: Product category (e.g., Electronics, Clothing, Home Decor).
- quantity: Number of units sold.
- price_per_unit: Price per unit of the product.
- tags: Tags assigned to the product (e.g., [“gift”, “discounted”]).
- gender: Gender of the customer (Male, Female, or Non-binary).
- total_sales: Computed as quantity * price_per_unit.

# 1. Detect Outliers with `.quantile()`

In [4]:
# Calculate IQR
Q1 = df['total_sales'].quantile(0.25)
Q3 = df['total_sales'].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers
outliers = df[(df['total_sales'] < Q1 - 1 * IQR) | (df['total_sales'] > Q3 + 1.5 * IQR)]
df_no_outliers = df[~df.index.isin(outliers.index)]  # Exclude rows matching outliers

In [5]:
print("\nOutliers:")
outliers


Outliers:


,order_id,customer_id,order_date,region,category,quantity,price_per_unit,tags,gender,total_sales
0,1,100,2025-01-10,North,Clothing,2,67,"['premium', 'gift']",Non-binary,10000
2,3,109,2025-01-14,North,Electronics,5,170,"['premium', 'gift']",Non-binary,15000


*Find more information in the official [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html)*

# 2. Unpack Nested Data with `.explode()`

In [6]:
# Ensure it is list type and not string
df['tags'] = df['tags'].apply(ast.literal_eval)

# Explode the nested data
exploded_df = df.explode('tags')

In [7]:
print("\nExploded Data:")
exploded_df.head()


Exploded Data:


,order_id,customer_id,order_date,region,category,quantity,price_per_unit,tags,gender,total_sales
0,1,100,2025-01-10,North,Clothing,2,67,premium,Non-binary,10000
0,1,100,2025-01-10,North,Clothing,2,67,gift,Non-binary,10000
1,2,104,2025-01-12,West,Clothing,2,139,premium,Female,5
1,2,104,2025-01-12,West,Clothing,2,139,gift,Female,5
2,3,109,2025-01-14,North,Electronics,5,170,premium,Non-binary,15000


*Find more information in the official [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html)*

# 3. Simplify Aggregations with `.agg()`

In [8]:
# Aggregations
summary = df.groupby('region')['total_sales'].agg(['mean', 'sum', 'max'])

print("Aggregated Data:")
print(summary)

Aggregated Data:
          mean    sum    max
region                      
East     560.0   1120    790
North   8432.0  25296  15000
West     193.4    967    604


*Find more information in the official [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)*

# 4. Streamline Transformations with `.pipe()`

In [9]:
# Define custom transformation functions
def fill_missing(df):
    # Explicitly assign the modified column back to the DataFrame
    df['quantity'] = df['quantity'].fillna(df['quantity'].median())
    return df

def normalize(df):
    # Explicitly assign the new column to the DataFrame
    df['sales_normalized'] = (df['total_sales'] - df['total_sales'].mean()) / df['total_sales'].std()
    return df

def add_features(df):
    # Explicitly assign the new column to the DataFrame
    df['sales_category'] = pd.cut(df['total_sales'], bins=[0, 500, 1000, float('inf')],
                                  labels=['Low', 'Medium', 'High'])
    return df

# Apply transformations using pipe
df = (df.pipe(fill_missing)
          .pipe(normalize)
          .pipe(add_features))

In [10]:
print("Transformed Data:")
df.head()

Transformed Data:


,order_id,customer_id,order_date,region,category,quantity,price_per_unit,tags,gender,total_sales,sales_normalized,sales_category
0,1,100,2025-01-10,North,Clothing,2,67,"[premium, gift]",Non-binary,10000,1.374344,High
1,2,104,2025-01-12,West,Clothing,2,139,"[premium, gift]",Female,5,-0.517302,Low
2,3,109,2025-01-14,North,Electronics,5,170,"[premium, gift]",Non-binary,15000,2.320640,High
3,4,101,2025-01-16,East,Home Decor,5,158,[discounted],Male,790,-0.368734,Medium
4,5,102,2025-01-18,West,Home Decor,2,112,"[premium, gift]",Female,224,-0.475855,Low


*Find more information in the official [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html)*

# 5. Analyze Relationships with `.crosstab()`

In [11]:
# Crosstab
crosstab_result = pd.crosstab(df['gender'], df['category'], margins=True)

In [12]:
print("Crosstab Result:")
print(crosstab_result)

Crosstab Result:
category    Clothing  Electronics  Home Decor  All
gender                                            
Female             2            0           1    3
Male               0            1           3    4
Non-binary         2            1           0    3
All                4            2           4   10


*Find more information in the official [documentation](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html)*